In [1]:
import numpy as np
import asyncio
import pandas as pd
import sqlite3
import json
import pickle
from datetime import datetime, timedelta, date
from collections import namedtuple, defaultdict
from typing import Tuple
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from typing import List, Tuple, Dict
from sklearn.compose import ColumnTransformer
import warnings
import plotly.express as px
import plotly.graph_objects as go
from xgboost import XGBRegressor
from copy import deepcopy
import os
import sys
import copy
import yfinance_ez as yf

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from finance_ml.variants.linear_model.main import main
from finance_ml.variants.linear_model.hyperparams import Hyperparams
from finance_ml.variants.linear_model.preprocessing import (apply_engineered_columns,
    preprocess_data, preprocess_quarterly_data, preprocess_stockpup_data, read_stock_info,
    compare_to_market_indices)
from finance_ml.utils.constants import (
    QuarterlyColumns, StockPupColumns, STOCKPUP_TABLE_NAME, QUARTERLY_DB_FILE_PATH,
    YF_QUARTERLY_TABLE_NAME, INDEX_COLUMNS, MISSING_SECTOR, MISSING_INDUSTRY,
    STOCK_GENERAL_INFO_CSV, FORMULAE, Q_DELTA_PREFIX, YOY_DELTA_PREFIX,
    QUARTER, YEAR, VS_MKT_IDX, CATEGORICAL_COLUMNS
)

warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

/Users/ezras/projects/personal/finance_ml/finance_ml/variants/linear_model/metamodel.py:57: SyntaxWarning:

assertion is always true, perhaps remove parentheses?



In [5]:
# y_pred, df = await main(hp)
df = pd.read_parquet('/Users/ezras/projects/personal/finance_ml/finance_ml/notebooks/data/processed_210415.parquet')

latest = '/Users/ezras/projects/personal/finance_ml/finance_ml/models/linear-model-2021-04-15.pickle'

with open(latest, 'rb') as f:
    metamodel = pickle.load(f)

hp = metamodel.hyperparams
model = metamodel.model

In [13]:
from finance_ml.variants.linear_model.backtest import compute_performance

loop = asyncio.get_event_loop()

await compute_performance(df=df,
                          start_date=date(2010,1,1),
                          hyperparams=hp,
                          end_date=None)

/Users/ezras/.pyenv/versions/3.7.0/lib/python3.7/inspect.py:732: RuntimeWarning:

coroutine 'compute_performance' was never awaited



AttributeError: 'QuarterlyIndex' object has no attribute 'day'